In [210]:
#%load_ext autoreload
#%autoreload 2
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm, metrics, tree, decomposition, svm


[autoreload of edited_pipeline failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 376, in superreload
    module = reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_cod

In [211]:
import edited_pipeline as ep
import pipeline as p_l

In [86]:
df = pd.DataFrame(np.random.randn(10000, 2))
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
y_pred = np.random.uniform(0,1,2000)
y_true= np.random.randint(0,2, size=2000)

In [170]:
data = p_l.read_data('/Users/csolisu/Downloads/projects_2012_2013.csv')

In [97]:
def roc_auc_sc(y_test_sorted,y_pred_probs_sorted):
    '''
    Gives the auc score 
    Input:
        y_test_sorted(array): true label from testing data
        y_pred_probs_sorted: predicted probabilities for label
    Returns:
        auc score
    '''
    return metrics.roc_auc_score(y_test_sorted,y_pred_probs_sorted)

def joint_sort_descending(l1, l2):
    # l1 and l2 have to be numpy arrays

    '''
    Order the specified arrays into descending (see which ones got the highest score)

    Intputs: 
        l1: first array
        l2: second array
    Returns: 
        The sorted data
    '''
    idx = np.argsort(l1)[::-1]
    return l1[idx], l2[idx]

def generate_binary_at_k(y_scores, k):
    '''
    Makes the cutoff for keeping only the observations that are within the top k%
    Input:
        y_scores: Predicted label
        k: threshold for keeping the points
    '''
    cutoff_index = int(len(y_scores) * (k / 100.0))
    test_predictions_binary = [1 if x < cutoff_index else 0 for x in range(len(y_scores))]
    return test_predictions_binary

def precision_at_k(y_true, y_scores, k):
    y_scores, y_true = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores, k)
    #precision, _, _, _ = metrics.precision_recall_fscore_support(y_true, preds_at_k)
    #precision = precision[1]  # only interested in precision for label 1
    precision = metrics.precision_score(y_true, preds_at_k)
    return precision

def recall_at_k(y_true, y_scores, k):
    #y_scores_sorted, y_true_sorted = zip(*sorted(zip(y_scores, y_true), reverse=True))
    y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores_sorted, k)
    #precision, _, _, _ = metrics.precision_recall_fscore_support(y_true, preds_at_k)
    #precision = precision[1]  # only interested in precision for label 1
    recall = metrics.recall_score(y_true_sorted, preds_at_k)
    return recall

def f1_at_k(y_true, y_scores, k):
    '''
    Calculates the F1 score of a determined k% testing sample

    Inputs:
        y_true: label from test data
        y_scores: predicted label for test data
        k: k%, the % value we want to predict

    Returns: 
        F1 score  
    '''
    y_scores_sorted, y_true_sorted = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores_sorted, k)
    recall = metrics.recall_score(y_true_sorted, preds_at_k)
    precision = metrics.precision_score(y_true_sorted, preds_at_k)
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

def accurate(y_test_sorted_,y_pred_sorted):
    '''
    Calculates the accuracy score of a determined k% testing sample

    Inputs:
        y_test_sorted: label from test data
        y_pred_sorted: predicted label for test data

    Returns: 
        accuracy score  
    '''

    accure = metrics.accuracy_score(y_test_sorted_,y_pred_sorted)
    return accure

def plot_precision_recall_n(y_true, y_prob, model_name):

    '''
    Plots the precision-recall sample for the specified model

    Inputs:
        y_test_sorted_: label from test data
        y_pred_sorted: predicted label for test data

    Returns: 
        Image with the precision recall curve
    '''

    from sklearn.metrics import precision_recall_curve
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax1.set_ylim([0,1])
    ax1.set_ylim([0,1])
    ax2.set_xlim([0,1])
    
    name = model_name
    plt.title(name)
    #plt.savefig(name)
    plt.show()




def plot_roc(name, probs, true, output_type):
    '''
    Plots the AUC for the specified model

    Input:
        name(string): name we want to assign to the model
        probs(array): predicted label for test data
        true(array): label for true data
        output_types()
    
    '''
    fpr, tpr, thresholds = roc_curve(true, probs)
    roc_auc = auc(fpr, tpr)
    pl.clf()
    pl.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    pl.plot([0, 1], [0, 1], 'k--')
    pl.xlim([0.0, 1.05])
    pl.ylim([0.0, 1.05])
    pl.xlabel('False Positive Rate')
    pl.ylabel('True Positive Rate')
    pl.title(name)
    pl.legend(loc="lower right")
    if (output_type == 'save'):
        plt.savefig(name)
    elif (output_type == 'show'):
        plt.show()
    else:
        plt.show()

In [109]:
def evaluate_models(metric_lst, k_lst,y_test_sorted,y_pred_probs_sorted):
    '''
    Creates a list of the evaluation metrics especified. 

    Inputs: 
        metric_lst (list)
        precision_lst(list)
        y_test_sorted(array)
        y_pred_probs(array)

    '''
    final_lst = []
    mtr_lst = metric_lst
    if roc_auc_sc in metric_lst:
        mtr_lst.remove(roc_auc_sc)
        final_lst.append(roc_auc_sc(y_test_sorted,y_pred_probs_sorted))
        
    for metr in mtr_lst:
        for prec in k_lst:
            final_lst.append(metr(y_test_sorted,y_pred_probs_sorted,prec))

    return final_lst

In [110]:
metric_lst =[roc_auc_sc,precision_at_k,recall_at_k,f1_at_k]
k_lst = [1.0,2.0,5.0,10.0,20.0,30.0,40.0,50.0]

In [111]:
lst = evaluate_models(metric_lst,k_lst,y_true,y_pred)

In [115]:
lst

[0.48596499495186984,
 0.6,
 0.575,
 0.54,
 0.525,
 0.52,
 0.5233333333333333,
 0.525,
 0.521,
 0.011417697431018078,
 0.021883920076117985,
 0.05137963843958135,
 0.09990485252140818,
 0.19790675547098002,
 0.2987630827783064,
 0.39961941008563273,
 0.49571836346336823,
 0.022408963585434174,
 0.042163153070577455,
 0.09383145091225022,
 0.16786570743405274,
 0.2866988283942109,
 0.38037552998182916,
 0.45380875202593196,
 0.5080448561677231]

In [136]:
metric_lst =[roc_auc_sc,precision_at_k,recall_at_k,f1_at_k]
k_lst = [1.0,2.0,5.0,10.0,20.0,30.0,40.0,50.0]

In [137]:
columns = ('train_end_date','model_type','clf', 'parameters')

In [138]:
def define_colnames(metric_lst,k_lst):
    final_lst = []
    names = {roc_auc_sc:'roc_auc',precision_at_k:'p_at_',recall_at_k: 'r_at_',f1_at_k: 'f1_at_'}
    mtr_lst = metric_lst
    if roc_auc_sc in metric_lst:
        mtr_lst.remove(roc_auc_sc)
        final_lst.append(names[roc_auc_sc])
    print(final_lst)
    for mtr in mtr_lst:
        for k in k_lst:
            final_lst.append(names[mtr] +'{}'.format(k))
    return final_lst
            

In [139]:
lsls = define_colnames(metric_lst,k_lst)

['roc_auc']


In [140]:
lsls

['roc_auc',
 'p_at_1.0',
 'p_at_2.0',
 'p_at_5.0',
 'p_at_10.0',
 'p_at_20.0',
 'p_at_30.0',
 'p_at_40.0',
 'p_at_50.0',
 'r_at_1.0',
 'r_at_2.0',
 'r_at_5.0',
 'r_at_10.0',
 'r_at_20.0',
 'r_at_30.0',
 'r_at_40.0',
 'r_at_50.0',
 'f1_at_1.0',
 'f1_at_2.0',
 'f1_at_5.0',
 'f1_at_10.0',
 'f1_at_20.0',
 'f1_at_30.0',
 'f1_at_40.0',
 'f1_at_50.0']

In [150]:
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta


In [161]:
def temp_val(start_time, end_time, window_train, window_test,less_days):
    '''
    Creates a list that especifies how the temporal windows should be
    Inputs:
        start_time(str): start time of dataset
        end_time(str): end time of dataset
        window_train(int): months for amplifyhing training dataset
        window_test(int): months for amplifyhing testing dataset
    Returns: 
        list of validated dates
    '''
    validation_lst = []
    start_time_date = datetime.strptime(start_time, '%Y-%m-%d')
    end_time_date = datetime.strptime(end_time, '%Y-%m-%d')
    test_end_time = end_time_date - relativedelta(days=+1)
    train_start_time = start_time_date
    while test_end_time <= end_time_date:
        train_end_time = train_start_time + relativedelta(months=+window_train) - relativedelta(days=+less_days)
        test_start_time = train_end_time + relativedelta(days=+less_days) + relativedelta(days=+1)
        test_end_time = test_start_time + relativedelta(months=+window_test)
        validation_lst.append([train_start_time,train_end_time,test_start_time,test_end_time])
        window_train +=6
    return validation_lst

In [165]:
validation_elem = temp_val('2012-01-01','2013-12-31',6,6,60)

In [178]:
def temp_spl(data,temp_var,validation_elem, label):

    '''
    creates training and testing sets based on temporal validtion lists
    Inputs:
        data(DataFrame): data we want to work with
        temp_va(str): feature that indicates time
        validation_elem(list): list of training start time, end time, testing start time, end time
        label(str): label name 
    Returns: 
         training and testing sets
    '''

    train_start,train_end,test_start,test_end = validation_elem
    data[temp_var] = pd.to_datetime(data[temp_var])
    train_data = data[(train_start <= data[temp_var]) & ( data[temp_var] <= train_end)]
    X_train = np.array(train_data.drop([label,temp_var],1))
    y_train = np.array(train_data[label])
    test_data = data[(test_start <= data[temp_var] ) & (data[temp_var]<= test_end)]
    X_test = np.array(test_data.drop([label,temp_var],1))
    y_test = np.array(test_data[label])
    return X_train, X_test, y_train, y_test 

def temp_spl_pd(data,temp_var,validation_elem, label):

    '''
    creates training and testing sets based on temporal validtion lists
    Inputs:
        data(DataFrame): data we want to work with
        temp_va(str): feature that indicates time
        validation_elem(list): list of training start time, end time, testing start time, end time
        label(str): label name 
    Returns: 
         training and testing sets
    '''

    train_start,train_end,test_start,test_end = validation_elem
    data[temp_var] = pd.to_datetime(data[temp_var])
    train_data = data[(train_start <= data[temp_var]) & ( data[temp_var] <= train_end)]
    X_train = train_data.drop([label,temp_var],1)
    y_train = train_data[label]
    test_data = data[(test_start <= data[temp_var] ) & (data[temp_var]<= test_end)]
    X_test = test_data.drop([label,temp_var],1)
    y_test = test_data[label]
    return X_train, X_test, y_train, y_test 

In [171]:
data['days_financed'] = pd.to_datetime(data['datefullyfunded']) - pd.to_datetime(data['date_posted'])
data['label'] = data['days_financed'].dt.days.apply(lambda x: 0 if x <= 60 else 1)

In [230]:
X_train, X_test, y_train, y_test = temp_spl_pd(data,'date_posted',validation_elem[1],'label')

In [225]:
def clean_data(data,quantile):
    p_l.drop_features(data, DROPS)
    p_l.drop_features(data, UNNECESARY)
    p_l.fill_missing(data,TO_FILL,form = False)
    p_l.features_quantile(data,TO_QUANTILE,quantile)
    p_l.drop_features(data, TO_QUANTILE)
    data = p_l.to_dummies(data,TO_DUMMIES)
    p_l.drop_features(data, TO_DUMMIES)
    return data

    

In [209]:
#Features we want to turn into quantiles
TO_QUANTILE = ['total_price_including_optional_support']
#Features we want to turn into dummies for their categorical nature
TO_DUMMIES = ['school_metro', 
               'primary_focus_area', 'resource_type', 'poverty_level',
              'grade_level']


#Features we want to drop, mainly the ids, latitude and longitude

DROPS = ['projectid', 'teacher_acctid', 'schoolid',
            'school_ncesid', 'school_latitude', 
            'school_longitude', 'school_district', 
            'school_county', 'secondary_focus_subject',
            'secondary_focus_area','primary_focus_subject','school_state']

#Features we want to impute
TO_FILL = ['primary_focus_area','resource_type', 'grade_level', 'students_reached']
#Features we already used
UNNECESARY = ['datefullyfunded', 'days_financed']
clean_data(X_train,10)

In [193]:
X_train.columns

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_metro', 'school_district', 'school_county', 'school_charter',
       'school_magnet', 'teacher_prefix', 'primary_focus_subject',
       'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area',
       'resource_type', 'poverty_level', 'grade_level', 'students_reached',
       'eligible_double_your_impact_match', 'datefullyfunded', 'days_financed',
       'total_price_including_optional_support_q10'],
      dtype='object')

In [232]:
for df_t in [X_train,X_test]:
    df_t = clean_data(df_t,10)

In [233]:
X_train.dtypes

school_city                                    object
school_metro                                   object
school_charter                                 object
school_magnet                                  object
teacher_prefix                                 object
primary_focus_area                             object
resource_type                                  object
poverty_level                                  object
grade_level                                    object
students_reached                              float64
eligible_double_your_impact_match              object
total_price_including_optional_support_q10      int64
dtype: object

In [229]:
X_train

,school_city,school_charter,school_magnet,teacher_prefix,students_reached,eligible_double_your_impact_match,total_price_including_optional_support_q10,school_metro_rural,school_metro_suburban,school_metro_urban,...,resource_type_Trips,resource_type_Visitors,poverty_level_high poverty,poverty_level_highest poverty,poverty_level_low poverty,poverty_level_moderate poverty,grade_level_Grades 3-5,grade_level_Grades 6-8,grade_level_Grades 9-12,grade_level_Grades PreK-2
1,Ventura,f,f,Mrs.,28.0,t,1,0,0,1,...,0,0,0,1,0,0,1,0,0,0
2,Los Angeles,f,f,Ms.,56.0,f,9,0,0,1,...,0,0,1,0,0,0,1,0,0,0
3,Brooklyn,f,t,Ms.,23.0,f,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
9,Newhall,f,f,Ms.,24.0,f,9,0,1,0,...,0,0,0,1,0,0,0,0,0,1
16,Vine Grove,f,f,Ms.,120.0,f,4,0,1,0,...,0,0,1,0,0,0,0,1,0,0
18,Van Nuys,f,f,Ms.,30.0,t,1,0,0,1,...,0,0,0,1,0,0,1,0,0,0
23,El Monte,f,f,Mrs.,30.0,f,9,0,1,0,...,0,0,0,1,0,0,0,0,0,1
25,Indianapolis,f,f,Mrs.,27.0,f,5,0,0,1,...,0,0,0,1,0,0,0,0,0,1
26,Tarboro,t,f,Mrs.,50.0,f,2,1,0,0,...,0,0,1,0,0,0,0,1,0,0
27,North Hills,f,f,Mr.,29.0,f,8,0,0,1,...,0,0,0,1,0,0,1,0,0,0
